In [13]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

from hydrad_tools.configure import Configure
from hydrad_tools.parse import Strand

%matplotlib inline

# `hydrad_tools` Demo

## Configuring a Simulation

In [2]:
config = Configure.load_config('/home/wbarnes/Documents/codes/hydrad_tools/defaults.asdf')

In [3]:
config.keys()

dict_keys(['asdf_library', 'history', 'general', 'grid', 'heating', 'initial_conditions', 'radiation', 'solver'])

In [4]:
config['general']

{'footpoint_height': <Quantity 5.e+08 cm>,
 'loop_inclination': <Quantity 0. deg>,
 'force_single_fluid': False,
 'heat_flux_timestep_limit': <Quantity 1.e-10 s>,
 'logging_frequency': 1000,
 'minimum_collisional_coupling_timescale': <Quantity 0.01 s>,
 'output_interval': <Quantity 1. s>,
 'heat_flux_limiting_coefficient': 0.167,
 'use_kinetic_model': False,
 'write_file_equation_terms': False,
 'write_file_hydrogen_level_populations': False,
 'write_file_ion_populations': False,
 'write_file_physical': True,
 'write_file_timescales': False}

In [5]:
config['general']['total_time'] = 5e3 * u.s
config['general']['loop_length'] = 9e9 * u.cm
# Equivalently
config['general']['total_time'] = 1.39 * u.h
config['general']['loop_length'] = 90 * u.Mm

In [6]:
config['initial_conditions']['heating_location'] = 45 * u.Mm
config['initial_conditions']['heating_scale_height'] = 1e300 * u.cm

In [7]:
config['grid']['maximum_refinement_level'] = 5
config['solver']['cutoff_temperature_fraction'] = 0.2

In [8]:
config['heating']['events'] = [
    {'time_start': 0*u.s,
     'total_duration': 200*u.s,
     'rise_duration': 100*u.s,
     'decay_duration': 100*u.s,
     'rate': 0.1 * u.erg / (u.cm**3) / u.s,
     'location': 45 * u.Mm,
     'scale_height': 1e300*u.cm}
]

In [9]:
c = Configure(config)

In [10]:
%%bash
rm -rf /home/wbarnes/Desktop/issi_test_demo

In [11]:
c.setup_simulation('/home/wbarnes/Desktop/',
                   base_path='/home/wbarnes/Documents/codes/HYDRAD/',
                   name='issi_test_demo',
                   verbose=False)

In [12]:
!ls /home/wbarnes/Desktop/issi_test_demo/

Forward_Model  HYDRAD_GUI		 Initial_Conditions.exe  README.md
Heating_Model  HYDRAD_GUI.jar		 Kinetic_Model		 Resources
HYDRAD	       hydrad_tools_config.asdf  LICENSE		 Results
HYDRAD.exe     Initial_Conditions	 Radiation_Model	 Visualisation


What if I pass a parameter with the wrong units?

In [29]:
config['general']['total_time'] = 2 * u.kg

In [31]:
Configure(config).setup_simulation(
    '/home/wbarnes/Desktop/',
    base_path='/home/wbarnes/Documents/codes/HYDRAD/',
    verbose=False
)

UnitConversionError: 'kg' (mass) and 's' (time) are not convertible

What if I don't include a required parameter?

In [32]:
config['general']['total_time'] = 1e3 * u.s
del config['general']['loop_length']

In [33]:
Configure(config).setup_simulation(
    '/home/wbarnes/Desktop/',
    base_path='/home/wbarnes/Documents/codes/HYDRAD/',
    verbose=False
)

MissingParameter: Parameter required for configuration

## Parsing Results

In [23]:
s = Strand('/home/wbarnes/Desktop/issi_test_demo/')

In [24]:
s.loop_length

<Quantity 9.e+09 cm>

In [25]:
s.time

<Quantity [0.000e+00, 1.000e+00, 2.000e+00, ..., 5.002e+03, 5.003e+03,
           5.004e+03] s>

In [22]:
s[:2]

TypeError: '<' not supported between instances of 'slice' and 'int'

## Visualizing Results